In [ ]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
tf.compat.v1.set_random_seed(1)
#sess = tf.Session(graph=tf.get_default_graph())
#K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import pickle
from video_process_utils import *
import collections

In [ ]:
target_col = "SEMLS_dev_residual"

In [ ]:
#assign train/validation/test ids
alldata_processed =\
    pd.read_csv("./data/processed/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed.groupby(['videoid','side']).head(1)
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [ ]:
datasplit_df = pd.read_csv('./data/processed/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [ ]:
with open('./data/processed/all_processed_video_segments.pickle', 'rb') as handle:
    processed_video_segments = pickle.load(handle)

In [ ]:
x_columns = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,
                  2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,50,51,52,53,54,55,56]

target_dict = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    target_dict[row['videoid']] = row[target_col]

In [ ]:
if target_col == "gmfcs":
    processed_video_segments = list(filter(lambda x: target_dict[x[0]] in range(1,6), processed_video_segments))

In [ ]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)[:,:,x_columns]

In [ ]:
y = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in all_ids])

In [ ]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)[:,:,x_columns]

In [ ]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)[:,:,x_columns]

In [ ]:
y_train = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in train_ids])

In [ ]:
y_validation = np.array([target_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids])

In [ ]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [ ]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [ ]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)
print(target_min, target_range)

In [ ]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)

y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights]))

In [ ]:
vid_length = 124

In [ ]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        #multiply by len(weights) to make the magnitude invariant to number of components in target
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences of course, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]
normal_weights = [1.0,0]


#normalize weights to sum to 1 to prevent affecting loss function
weights = weights/np.sum(weights)
normal_weights = normal_weights/np.sum(normal_weights)

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#monitor our actual objective
mse_metric = w_mse(target_range**2*normal_weights)

hyper_str = "params_"
for param in [initial_lrate,epochs_drop,drop_factor,dropout_amount,conv_dim,last_layer_dim,filter_length,l2_lambda]:
    hyper_str = hyper_str + str(param) + "_"

K.clear_session()
#K.set_session(sess)

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

In [ ]:
checkpoint_folder = "./data/checkpoints/cnn_checkpoints_%s" % (target_col)

In [ ]:
from keras.callbacks import LearningRateScheduler

from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN

train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_1:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)

    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_2', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=32, epochs=n_epochs,shuffle=True)

In [ ]:
import statsmodels.api as sm

In [ ]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [ ]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [ ]:
def predict_and_aggregate_singlevar(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    df["videoid"] = np.array(videoids)
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [ ]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
predictions_df[target_col] = y
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [ ]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

In [ ]:
predictions_df.groupby(['videoid','dataset'],as_index=False).mean().to_csv("./data/predictions/cnn_%s_singlesided_predictions_all_epochs.csv" % (target_col),index=False)

In [ ]:
# Save best models
# This must be run after finding the best model with select_optimal_epoch 
maps = {
    "gmfcs": "./data/checkpoints/cnn_checkpoints_gmfcs/weights-08-0.5025.hdf5", #
    "speed": "./data/checkpoints/cnn_checkpoints_speed/weights-77-0.0336.hdf5", #
    "cadence": "./data/checkpoints/cnn_checkpoints_cadence/weights-36-0.0211.hdf5", #
    "SEMLS_dev_residual": "./data/checkpoints/cnn_checkpoints_SEMLS_dev_residual/weights-32-0.8929.hdf5", #
#    "GDI": "./data/checkpoints/cnn_checkpoints_GDI/weights-88-72.0330.hdf5" #
    "GDI": "./data/checkpoints/cnn_checkpoints_GDI/weights-92-90.8354.hdf5" #
}
for col in maps.keys():
    model_folder_path = "./data/models/%s_best.pb" % (col)
    model.load_weights(maps[col])
    model.save(model_folder_path)